In [9]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import glob

In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox

## Select folders

In [11]:
root = tk.Tk()
root.withdraw()
messagebox.showinfo("Folder Selection", "Press OK and then select the master folder (raw images)")

photos_dir_master = filedialog.askdirectory(title="Select the master folder")
photos_dir_master

'/Users/semenzalab/Desktop/ImageJ_cellcounting/demo/trial'

In [12]:
# folder containing csv files with results from ImageJ
messagebox.showinfo("Folder Selection", "Press OK and then select the results folder (containing all csv files to be analyzed)")
results_dir = filedialog.askdirectory(title="Select the folder containing resulting csv files from ImageJ")
results_dir

'/Users/semenzalab/Desktop/ImageJ_cellcounting/demo/trial_output/Results'

# Run cells below

In [13]:
# folder path you want to store processed results in
output_dir = os.path.join(os.path.dirname(results_dir), "Processed Results 2")
os.makedirs(output_dir)

In [14]:
images_per_figure = []
figures = [folder for folder in os.listdir(photos_dir_master) if os.path.isdir(os.path.join(photos_dir_master, folder))]

In [15]:
results = glob.glob(os.path.join(results_dir, "*.csv"))

In [16]:
cellarea_dct_sq = {}
for i in range(len(figures)):
    csv_filename = os.path.join(results_dir, figures[i] + ".csv")
    cellnums = pd.read_csv(csv_filename, index_col=0)
    image_names_i = glob.glob(os.path.join(photos_dir_master + "/" + figures[i], "**", "*.tif"), recursive=True)
    image_names_i = [j.split("/")[-1] for j in image_names_i]
    cellarea_dct = {j: sum(cellnums.loc[cellnums["Label"] == f"outliers_{j}", "Area"]) for j in image_names_i}
    cellarea_dct_sq.update({figures[i]: cellarea_dct})

In [17]:
cellarea_dct_sq

{'f2': {'ImageD3-1.tif': 522852,
  'ImageD1-3.tif': 488775,
  'ImageD1-2.tif': 272361,
  'ImageD3-2.tif': 329275,
  'ImageD1-1.tif': 665311,
  'ImageD3-3.tif': 465457,
  'ImageD2-2.tif': 258666,
  'ImageD2-3.tif': 908363,
  'ImageD2-1.tif': 564072,
  'ImageC2-2.tif': 3627645,
  'ImageC2-3.tif': 3138201,
  'ImageC2-1.tif': 2065779,
  'ImageC3-1.tif': 3712663,
  'ImageC1-3.tif': 1143992,
  'ImageC1-2.tif': 1299569,
  'ImageC3-2.tif': 4853226,
  'ImageC1-1.tif': 1147888,
  'ImageC3-3.tif': 3595306},
 'f1': {'ImageB2-1.tif': 719644,
  'ImageB2-3.tif': 939556,
  'ImageB2-2.tif': 1146521,
  'ImageB3-3.tif': 1285514,
  'ImageB1-1.tif': 473153,
  'ImageB3-2.tif': 2154828,
  'ImageB1-2.tif': 698524,
  'ImageB1-3.tif': 986761,
  'ImageB3-1.tif': 1142007,
  'ImageA2-1.tif': 251623,
  'ImageA2-2.tif': 382933,
  'ImageA2-3.tif': 346899,
  'ImageA3-2.tif': 764688,
  'ImageA3-3.tif': 612868,
  'ImageA1-1.tif': 471271,
  'ImageA1-3.tif': 452152,
  'ImageA3-1.tif': 535606,
  'ImageA1-2.tif': 461265}}

In [20]:
names = [f"{i}-{j}" for i in range(1,4) for j in range(1,4)]
names_br = list(range(1,4))
image_size = 4080 * 3072

for folder in cellarea_dct_sq.keys():
    cellarea_df = pd.DataFrame.from_dict(cellarea_dct_sq[folder], orient="index").reset_index()
    cellarea_df.columns = ["image_name", "cell_area"]
    cellarea_df["cell_area"] = cellarea_df["cell_area"] / image_size
    
    
    df = cellarea_df.copy()
    df.set_index("image_name", inplace=True)
    reform_idx = [j.lower() for j in df.index]
    df.index = reform_idx
    print(df)
    bio_groups = np.unique([j[:-7] for j in reform_idx])
    new_idx = [f"{bio_group}{name}.tif" for bio_group in bio_groups for name in names]
    new_dct = {new_id: df.loc[new_id, "cell_area"] for new_id in new_idx}
    new_vals = np.array(list(new_dct.values())).reshape(-1,3)
    new_df = pd.DataFrame(new_vals, columns=["TR1", "TR2", "TR3"], 
                          index=[f"{bio_group}{k}" for bio_group in bio_groups for k in names_br]).\
                            reset_index()

    with pd.ExcelWriter(f"{output_dir}/{folder}_processed.xlsx") as f:
        cellarea_df.to_excel(f, sheet_name="cell_area")
        new_df.to_excel(f, sheet_name="re-formatted")

        

               cell_area
imaged3-1.tif   0.041715
imaged1-3.tif   0.038997
imaged1-2.tif   0.021730
imaged3-2.tif   0.026271
imaged1-1.tif   0.053082
imaged3-3.tif   0.037136
imaged2-2.tif   0.020638
imaged2-3.tif   0.072473
imaged2-1.tif   0.045004
imagec2-2.tif   0.289430
imagec2-3.tif   0.250380
imagec2-1.tif   0.164817
imagec3-1.tif   0.296213
imagec1-3.tif   0.091273
imagec1-2.tif   0.103685
imagec3-2.tif   0.387212
imagec1-1.tif   0.091584
imagec3-3.tif   0.286850
               cell_area
imageb2-1.tif   0.057416
imageb2-3.tif   0.074962
imageb2-2.tif   0.091475
imageb3-3.tif   0.102564
imageb1-1.tif   0.037750
imageb3-2.tif   0.171922
imageb1-2.tif   0.055731
imageb1-3.tif   0.078728
imageb3-1.tif   0.091114
imagea2-1.tif   0.020076
imagea2-2.tif   0.030552
imagea2-3.tif   0.027677
imagea3-2.tif   0.061010
imagea3-3.tif   0.048897
imagea1-1.tif   0.037600
imagea1-3.tif   0.036075
imagea3-1.tif   0.042733
imagea1-2.tif   0.036802


: 